In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=10
year=21
dept=6
source="PA"
time=30
time_type="HC"
# accessibilite_exp=-0.12 #0.08 pour 45
accessibilite_exp=-(75-time)*4/1500
print(year, dept, specialite, source, time, time_type, accessibilite_exp)

21 6 10 PA 30 HC -0.12


In [3]:
def get_ps(year, specialite):
    sql=f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join specialite s on s.id=sp.specialite_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
and ar.dept_id={dept}
group by ps.id, c.id, an.id, i.id
"""
    print(f"Quering PS for year {year} and specialite {specialite}")
    return pd.read_sql(sql, config.connection_string)

In [4]:
def get_pa(year, specialite):
    sql = f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat 
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and ps.code_mode_exercice='L'
and ds.annee={year}
and pands.adresse_norm_id is not null
and an.dept_id={dept}
group by pa.id, an.id, i.id
"""
    print(f"Quering PA for year {year} and specialite {specialite}")
    print(sql)
    return pd.read_sql(sql, config.connection_string)


In [5]:
def get_by_source(year, specialite, source):
    if source=="PS":
        return get_ps(year, specialite)
    elif source=="PA":
        return get_pa(year, specialite)
    else:
        raise ValueError(f"Bad source: {source}")
    

In [6]:
df = get_by_source(year, specialite, source)
df

Quering PA for year 21 and specialite 10

select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat 
from apl.ps_libreacces ps
join personne_activite pa on pa.inpp=ps.inpp
join pa_adresse_norm_date_source pands on pands.personne_activite_id=pa.id and pands.date_source_id=ps.date_source_id
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on d.code_diplome=ps.code_diplome
join profession_diplome pd on pd.diplome_id=d.id
join profession p on p.id=pd.profession_id
join specialite_profession sp on sp.profession_id=p.id
join adresse_norm an on an.id=pands.adresse_norm_id
join date_source ds on ds.id=pands.date_source_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
and ps.code_mode_exercice='L'
and ds.annee=21
and pands.adresse_norm_id is not null
and an.dept_id=6
group by pa.id, an.id, i.id



,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,3079,CAUVIN ZACCONI,ANNE MARIE,6,1116516,60881302,7.256556,43.719388
1,3568,LEDORE,VANINA,6,86364,60300110,7.008846,43.570037
2,3568,LEDORE,VANINA,6,1199693,60290114,7.005294,43.562818
3,4008,TIMAR,GABOR,6,75235,60850108,7.040872,43.588167
4,4008,TIMAR,GABOR,6,653057,60040108,7.120794,43.599494
...,...,...,...,...,...,...,...,...
1568,1539558,GUEDJ,JACQUES,6,932019,60881702,7.273915,43.703665
1569,1540045,CORALLINI TEISSIER,MARIE MADELEINE,6,63059,60881703,7.267766,43.706909
1570,1540048,DUBOIS,MICHEL,6,864208,61360000,7.450072,43.877190
1571,1540051,ATLANI,ERIC,6,395822,60290111,7.024949,43.552110


In [7]:
nb_ps = df["id"].nunique()
unique_ps = df.groupby(["id", "lon", "lat"])
print(f"Nb PS {nb_ps}")
print(f"Nb unique PS {len(unique_ps)}")

Nb PS 1281
Nb unique PS 1572


In [8]:
df["key"] = df["id"].astype(str) + "_" + df["lat"].astype(str) + "_" + df["lon"].astype(str)
df["nb_cabinet"] = df.groupby("id")["key"].transform("nunique")
df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1    0.679593
2    0.204704
3    0.061030
4    0.035601
5    0.019072
Name: proportion, dtype: float64

In [9]:
df["weight"] = 1 / df["nb_cabinet"]
df["nb"] = df.groupby("iris")["weight"].transform("sum")
df.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
0,3079,CAUVIN ZACCONI,ANNE MARIE,6,1116516,60881302,7.256556,43.719388,3079_43.719388_7.256556,1,1.0,8.000000
1,3568,LEDORE,VANINA,6,86364,60300110,7.008846,43.570037,3568_43.570037_7.008846,2,0.5,4.500000
2,3568,LEDORE,VANINA,6,1199693,60290114,7.005294,43.562818,3568_43.562818_7.005294,2,0.5,7.783333
3,4008,TIMAR,GABOR,6,75235,60850108,7.040872,43.588167,4008_43.588167_7.040872,2,0.5,1.500000
4,4008,TIMAR,GABOR,6,653057,60040108,7.120794,43.599494,4008_43.599494_7.120794,2,0.5,1.033333
5,4439,SERFATY SABOT,RACHEL,6,1534433,60882402,7.264646,43.697941,4439_43.697941_7.264646,1,1.0,12.500000
6,4673,PIONNEAU,AUDREY,6,272923,61120000,7.003135,43.673439,4673_43.673439_7.003135,1,1.0,6.500000
7,9987,OLLIVIER,JEAN YVES,6,828675,60882801,7.229759,43.679983,9987_43.679983_7.229759,1,1.0,8.500000
8,9993,GIOBERGIA,ROGER,6,680165,61230108,7.185653,43.668797,9993_43.668797_7.185653,1,1.0,2.533333
9,9997,SCEMAMA,ISABELLE,6,1210662,60290112,7.016651,43.559519,9997_43.559519_7.016651,1,1.0,6.000000


In [10]:
df = df.sort_values(by='iris')
df = df.drop_duplicates(subset=['iris', 'nb'])
df


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
800,690001,BILLAUD,JEAN CHRISTOPHE,6,507852,60040102,7.070016,43.597006,690001_43.597006_7.070016,1,1.0,3.000000
690,573116,DIAMANT,GUY,6,1572610,60040103,7.137517,43.567521,573116_43.567521_7.137517,1,1.0,2.000000
1565,1534638,MANDRILE,DIDIER,6,1651,60040104,7.092388,43.604533,1534638_43.604533_7.092388,1,1.0,2.000000
1450,1388268,COQUELET MOULINOUX,ANNE,6,623107,60040105,7.115366,43.593250,1388268_43.59325_7.115366,1,1.0,3.000000
594,486187,FRICKEY,SARAH,6,2007,60040106,7.112369,43.606074,486187_43.606074345716316_7.112369457663655,1,1.0,12.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
239,226731,CHEMOUNI,KARINE,6,84186,61610104,7.142204,43.642154,226731_43.642154_7.142204,1,1.0,1.000000
1489,1437614,BRUNET,DIDIER CLEMENT,6,1249056,61610105,7.134178,43.635682,1437614_43.635682_7.134178,1,1.0,6.833333
456,377059,DI VINCENZO,DOMINIQUE,6,182425,61620000,7.650077,44.087979,377059_44.087979_7.650077,5,0.2,0.200000
1464,1398844,GERSCHTEIN,JEAN LOUIS,6,1287593,61630000,7.593494,44.082422,1398844_44.082422_7.593494,2,0.5,2.150000


In [11]:
def get_iris_matrix(time_type: str):
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
where id / 10000000 = {dept}
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    m = pd.read_sql(sql, config.connection_string)
    m["time"] = m[f"time_{time_type.lower()}"].copy()
    return m

iris_matrix = get_iris_matrix(time_type)
# iris_matrix = pd.read_csv("od_06_sql_rename.csv")
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    


(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time_hc", 0 "time_hp" from iris.iris
where id / 10000000 = 6
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time_hc", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000=6 or iris_id_to/10000000=6)
and route_min <= 30) order by "iris1", "iris2"



,iris1,iris2,km,time_hc,time_hp,time,iris
0,40080000,60530000,17,22,22,22,60530000
1,40080000,60990000,21,24,24,24,60990000
2,40390000,61540000,24,30,30,30,61540000
3,40420000,60530000,11,19,19,19,60530000
4,40420000,60710000,22,30,32,30,60710000
...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,35,29,60850103
117765,831380000,60950101,18,19,21,19,60950101
117766,831380000,60950102,19,22,25,22,60950102
117767,831380000,60950103,18,20,23,20,60950103


In [12]:
def get_pop_iris():
    yy=min(21, year)
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'
and year={yy}
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

pop_iris = get_pop_iris()
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'
and year=21



,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,60120105,060120105,06012,A,3136.830308,68.356436,55.185368,179.895476,266.832339,226.432893,571.248108,831.040803,391.639115,351.412985,194.786785
1,60500000,060500000,06050,C,526.000000,14.188627,14.119226,40.517897,47.603916,17.493201,71.924387,145.187085,83.001180,65.334467,26.630014
2,60720000,060720000,06072,C,142.000000,0.000000,6.761905,4.829932,4.829932,3.863946,21.251701,27.047619,27.047619,31.877551,14.489796
3,61050101,061050101,06105,A,4007.649525,96.881691,145.234138,320.060918,454.509675,180.143151,475.809676,953.212078,561.152910,592.431015,228.214272
4,60150000,060150000,06015,C,1227.000000,35.622418,38.520557,52.247192,81.918133,62.907751,196.126186,266.670404,226.738245,194.355336,71.893777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,60290105,060290105,06029,A,1796.714284,44.260892,42.413851,104.320349,114.246691,193.954240,279.163799,364.078585,224.488425,293.639092,136.148359
496,60290112,060290112,06029,A,2338.574803,59.470820,48.421589,82.493068,164.290899,140.596745,363.755311,403.165863,266.097697,476.300304,333.982506
497,60290114,060290114,06029,A,2334.760199,49.321752,62.945962,64.563590,114.030958,120.238328,365.570798,456.005185,310.109098,485.178149,306.796378
498,60290119,060290119,06029,A,1576.416786,32.213206,48.717615,53.030786,104.965020,83.013723,194.164978,286.462978,257.549389,351.675755,164.623337


In [13]:
apl = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,40080000,60530000,17,22,22,22,60530000,060530000,06053,C,143.000000,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582
1,40080000,60990000,21,24,24,24,60990000,060990000,06099,C,1813.000000,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272
2,40390000,61540000,24,30,30,30,61540000,061540000,06154,C,495.000000,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879
3,40420000,60530000,11,19,19,19,60530000,060530000,06053,C,143.000000,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582
4,40420000,60710000,22,30,32,30,60710000,060710000,06071,C,584.000000,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,35,29,60850103,060850103,06085,A,2603.604864,57.428416,72.207652,135.459522,194.409719,141.793750,318.526897,479.908067,393.484524,562.460591,247.925725
117765,831380000,60950101,18,19,21,19,60950101,060950101,06095,A,3223.464405,77.910796,87.987431,210.729551,255.894450,155.595176,478.030563,685.139541,418.753123,589.352449,264.071324
117766,831380000,60950102,19,22,25,22,60950102,060950102,06095,A,2563.685976,67.975915,77.960010,155.903079,226.090400,145.568264,401.767992,529.572959,354.290116,427.490829,177.066411
117767,831380000,60950103,18,20,23,20,60950103,060950103,06095,A,2468.849619,46.976690,68.925850,125.728002,224.736092,103.186116,259.487677,537.700844,388.050677,528.083677,185.973993


In [14]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)

# apl["time"] = apl["time"].replace(0, 0.5)
apl["accessibilite_weight"] = accessibilite_fn(apl["time"])
apl.head(10)

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
0,40080000,60530000,17,22,22,22,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.071361
1,40080000,60990000,21,24,24,24,60990000,060990000,06099,C,1813.0,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135
2,40390000,61540000,24,30,30,30,61540000,061540000,06154,C,495.0,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879,0.027324
3,40420000,60530000,11,19,19,19,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284
4,40420000,60710000,22,30,32,30,60710000,060710000,06071,C,584.0,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769,0.027324
5,40420000,60990000,17,24,24,24,60990000,060990000,06099,C,1813.0,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135
6,40420000,61240000,15,30,30,30,61240000,061240000,06124,C,52.0,2.557377,0.852459,0.000000,0.000000,0.852459,6.819672,4.262295,7.672131,23.016393,5.967213,0.027324
7,40430000,60020000,15,20,20,20,60020000,060020000,06002,C,51.0,0.944444,0.000000,0.944444,0.000000,4.722222,2.833333,12.277778,12.277778,14.166667,2.833333,0.090718
8,40430000,60240000,8,11,11,11,60240000,060240000,06024,C,184.0,4.380952,5.476190,4.380952,16.428571,3.285714,19.714286,36.142857,20.809524,55.857143,17.523810,0.267135
9,40430000,60450000,18,23,23,23,60450000,060450000,06045,C,76.0,2.710965,0.903655,0.903655,1.807310,4.518275,11.840499,5.421930,17.169444,22.591374,8.132895,0.063292


In [15]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
105136,61290000,60730000,14,17,17,17,60730000,060730000,06073,C,653.0,15.966553,14.947243,42.905088,55.876926,20.971505,96.981393,145.940458,117.149194,107.190565,35.071074,0.130029
105137,61290000,60800000,10,18,19,18,60800000,060800000,06080,C,105.0,4.952830,1.981132,3.962264,3.962264,2.971698,10.896226,18.820755,16.839623,35.660377,4.952830,0.115325
105138,61290000,60820000,23,30,30,30,60820000,060820000,06082,C,115.0,0.833333,0.833333,2.500000,6.666667,5.833333,13.333333,30.000000,18.333333,25.000000,11.666667,0.027324
105139,61290000,61020000,13,21,21,21,61020000,061020000,06102,C,133.0,5.181818,1.727273,4.318182,10.363636,4.318182,12.954545,21.590909,31.954545,29.363636,11.227273,0.080460
105140,61290000,61100000,12,19,19,19,61100000,061100000,06110,C,116.0,2.924370,2.924370,2.924370,6.823529,4.873950,11.697479,24.369748,35.092437,16.571429,7.798319,0.102284
105141,61290000,61110000,10,17,17,17,61110000,061110000,06111,C,113.0,0.000000,0.000000,1.899160,3.798319,2.848739,15.193277,22.789916,26.588235,25.638655,14.243697,0.130029
105142,61290000,61290000,0,0,0,0,61290000,061290000,06129,C,301.0,1.104023,4.449030,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.768530,24.811635,1.000000
105143,61290000,61460000,18,23,23,23,61460000,061460000,06146,C,149.0,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292
105144,61290000,61530000,14,21,21,21,61530000,061530000,06153,C,835.0,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,165.201707,152.189294,156.194759,48.060321,0.080460


In [16]:
cols = [col for col in apl.columns if "pop" in col and col != "pop"]
cols

['pop0002',
 'pop0305',
 'pop0610',
 'pop1117',
 'pop1824',
 'pop2539',
 'pop4054',
 'pop5564',
 'pop6579',
 'pop80p']

In [17]:
yy=min(year, 24)
sql = f"""
select o.* from apl.overrepresentation o
where o.year={yy} and o.psp_spe_snds=1
"""
over = pd.read_sql(sql, config.connection_string)
# todo ajouter la colonne psp_spe_snds dans specialite
over

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,21,1,0.973677,0.4503,0.335766,0.350976,0.631042,0.859117,0.889063,1.115851,1.588182,2.69009


In [18]:
weights = over.values[0,2:]
weights

array([0.97367689, 0.45029979, 0.33576613, 0.35097573, 0.63104171,
       0.85911688, 0.8890631 , 1.11585109, 1.58818186, 2.6900896 ])

In [19]:
apl["pop_gp"] = sum(w * apl[c] for w, c in zip(weights, cols))
# if specialite == 5:
#     apl["pop_gp"] /= 2
apl.sort_values(by='iris2')
apl.head(5)   

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
0,40080000,60530000,17,22,22,22,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.071361,172.651676
1,40080000,60990000,21,24,24,24,60990000,060990000,06099,C,1813.0,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135,1918.295214
2,40390000,61540000,24,30,30,30,61540000,061540000,06154,C,495.0,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879,0.027324,546.713342
3,40420000,60530000,11,19,19,19,60530000,060530000,06053,C,143.0,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284,172.651676
4,40420000,60710000,22,30,32,30,60710000,060710000,06071,C,584.0,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769,0.027324,695.829940


In [20]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time_hc,time_hp,time,iris,iris_string,code_commune,type_iris,pop,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
105136,61290000,60730000,14,17,17,17,60730000,060730000,06073,C,653.0,15.966553,14.947243,42.905088,55.876926,20.971505,96.981393,145.940458,117.149194,107.190565,35.071074,0.130029,677.900545
105137,61290000,60800000,10,18,19,18,60800000,060800000,06080,C,105.0,4.952830,1.981132,3.962264,3.962264,2.971698,10.896226,18.820755,16.839623,35.660377,4.952830,0.115325,125.154081
105138,61290000,60820000,23,30,30,30,60820000,060820000,06082,C,115.0,0.833333,0.833333,2.500000,6.666667,5.833333,13.333333,30.000000,18.333333,25.000000,11.666667,0.027324,137.719957
105139,61290000,61020000,13,21,21,21,61020000,061020000,06102,C,133.0,5.181818,1.727273,4.318182,10.363636,4.318182,12.954545,21.590909,31.954545,29.363636,11.227273,0.080460,156.454272
105140,61290000,61100000,12,19,19,19,61100000,061100000,06110,C,116.0,2.924370,2.924370,2.924370,6.823529,4.873950,11.697479,24.369748,35.092437,16.571429,7.798319,0.102284,128.786996
105141,61290000,61110000,10,17,17,17,61110000,061110000,06111,C,113.0,0.000000,0.000000,1.899160,3.798319,2.848739,15.193277,22.789916,26.588235,25.638655,14.243697,0.130029,145.787120
105142,61290000,61290000,0,0,0,0,61290000,061290000,06129,C,301.0,1.104023,4.449030,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.768530,24.811635,1.000000,323.165658
105143,61290000,61460000,18,23,23,23,61460000,061460000,06146,C,149.0,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292,144.233423
105144,61290000,61530000,14,21,21,21,61530000,061530000,06153,C,835.0,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,165.201707,152.189294,156.194759,48.060321,0.080460,889.407371


In [21]:
# apl = apl.drop(["iris", "iris_string", "code_commune"], axis=1)
# apl = apl.drop(cols, axis=1)
apl = apl[["iris1","iris2","km","time","accessibilite_weight","pop_gp","pop","type_iris"]]
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris
0,40080000,60530000,17,22,0.071361,172.651676,143.0,C
1,40080000,60990000,21,24,0.056135,1918.295214,1813.0,C
2,40390000,61540000,24,30,0.027324,546.713342,495.0,C
3,40420000,60530000,11,19,0.102284,172.651676,143.0,C
4,40420000,60710000,22,30,0.027324,695.829940,584.0,C


In [22]:
apl["iris"] = apl["iris1"].copy()
apl.sort_values(by='iris2')
apl = apl.merge(df, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb
0,40080000,60530000,17,22,0.071361,172.651676,143.000000,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40080000,60990000,21,24,0.056135,1918.295214,1813.000000,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40390000,61540000,24,30,0.027324,546.713342,495.000000,C,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40420000,60530000,11,19,0.102284,172.651676,143.000000,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40420000,60710000,22,30,0.027324,695.829940,584.000000,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,0.030807,2991.248038,2603.604864,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117765,831380000,60950101,18,19,0.102284,3507.693226,3223.464405,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117766,831380000,60950102,19,22,0.071361,2691.432959,2563.685976,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117767,831380000,60950103,18,20,0.090718,2735.950887,2468.849619,A,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
apl["nb"] = apl["nb"].fillna(0)
apl = apl.sort_values(by=['iris1', "iris2"])
apl["nb"].unique()

array([ 0.        ,  3.        ,  2.        , 12.66666667, 10.66666667,
        1.03333333,  3.33333333, 12.83333333,  1.        ,  1.5       ,
        0.5       , 21.53333333, 10.        ,  4.        ,  3.5       ,
        4.5       ,  6.        , 11.        ,  7.        ,  1.33333333,
        2.2       , 12.5       , 15.        ,  5.25      ,  3.25      ,
        1.45      ,  5.33333333,  8.5       , 17.5       ,  7.78333333,
        1.25      ,  4.08333333,  5.5       ,  1.16666667,  7.5       ,
        0.25      ,  7.66666667,  7.25      ,  2.33333333,  6.83333333,
        4.16666667,  1.58333333,  1.66666667,  9.5       ,  2.5       ,
       15.2       ,  0.33333333,  5.        ,  2.66666667,  4.33333333,
       11.83333333,  4.75      ,  3.6       , 10.5       ,  3.08333333,
        8.4       , 21.73333333,  5.6       ,  6.91666667,  5.83333333,
        8.        ,  3.2       , 14.16666667,  9.7       , 24.93333333,
       10.33333333, 12.6       ,  4.83333333,  7.08333333,  5.66

In [24]:
apl = apl.sort_values(by='iris1')
apl["wpop"] = apl["accessibilite_weight"] * apl["pop_gp"]
apl["swpop"] = apl.groupby("iris1")["wpop"].transform("sum")
apl["R"] = apl["nb"] / (apl["swpop"] / 100000)
apl.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R
0,40080000,60530000,17,22,0.071361,172.651676,143.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.320643,120.003690,0.0
1,40080000,60990000,21,24,0.056135,1918.295214,1813.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,107.683047,120.003690,0.0
2,40390000,61540000,24,30,0.027324,546.713342,495.0,C,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.938244,14.938244,0.0
3,40420000,60530000,11,19,0.102284,172.651676,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.659540,146.383428,0.0
4,40420000,60710000,22,30,0.027324,695.829940,584.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,19.012664,146.383428,0.0


In [25]:
rgp = apl[apl["iris1"] == apl["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp", "pop"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp,pop
76,60010000,60010000,C,100.089449,0.000000,0.000000,299.573611,100.089449,100.089449,93.000000
87,60020000,60020000,C,61.387834,0.000000,0.000000,263.427392,61.387834,61.387834,51.000000
98,60030000,60030000,C,621.316785,0.000000,0.000000,1237.773641,621.316785,621.316785,640.000000
112,60040101,60040101,C,1699.147290,0.000000,0.000000,192073.227204,1699.147290,1699.147290,1763.364267
477,60040102,60040102,A,2527.038049,3.000000,1.954249,153511.652070,2527.038049,2527.038049,2534.265666
...,...,...,...,...,...,...,...,...,...,...
115797,61610105,61610105,A,4042.522443,6.833333,3.299546,207099.222109,4042.522443,4042.522443,3584.453834
116147,61610106,61610106,A,3868.825988,0.000000,0.000000,150318.655609,3868.825988,3868.825988,3539.143531
116501,61610107,61610107,A,2772.707609,0.000000,0.000000,164355.721307,2772.707609,2772.707609,2766.654836
116506,61620000,61620000,C,875.491796,0.200000,9.602474,2082.796507,875.491796,875.491796,719.000000


In [26]:
apl = apl.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest
0,40080000,60530000,17,22,0.071361,172.651676,143.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.320643,120.003690,0.0,60530000,C,172.651676,0.0,0.0,406.744412,172.651676,172.651676,143.0
1,40420000,60530000,11,19,0.102284,172.651676,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.659540,146.383428,0.0,60530000,C,172.651676,0.0,0.0,406.744412,172.651676,172.651676,143.0
2,40760000,60530000,15,19,0.102284,172.651676,143.0,C,40760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.659540,773.199207,0.0,60530000,C,172.651676,0.0,0.0,406.744412,172.651676,172.651676,143.0
3,40900000,60530000,22,28,0.034735,172.651676,143.0,C,40900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.997101,58.412067,0.0,60530000,C,172.651676,0.0,0.0,406.744412,172.651676,172.651676,143.0
4,41740000,60530000,14,20,0.090718,172.651676,143.0,C,41740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.662607,152.554588,0.0,60530000,C,172.651676,0.0,0.0,406.744412,172.651676,172.651676,143.0


In [27]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl.head(5)

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
0,40080000,60530000,17,22,0.071361,172.651676,143.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.320643,120.003690,0.0,60530000,C,172.651676,0.0,0.000000,406.744412,172.651676,172.651676,143.0,0.000000,7.369984
12,40080000,60990000,21,24,0.056135,1918.295214,1813.0,C,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,107.683047,120.003690,0.0,60990000,C,1918.295214,3.0,131.290903,2285.002180,1918.295214,1918.295214,1813.0,7.369984,7.369984
33,40390000,61540000,24,30,0.027324,546.713342,495.0,C,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.938244,14.938244,0.0,61540000,C,546.713342,1.0,101.410228,986.093825,546.713342,546.713342,495.0,2.770905,2.770905
1,40420000,60530000,11,19,0.102284,172.651676,143.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.659540,146.383428,0.0,60530000,C,172.651676,0.0,0.000000,406.744412,172.651676,172.651676,143.0,0.000000,9.571813
51,40420000,60710000,22,30,0.027324,695.829940,584.0,C,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,19.012664,146.383428,0.0,60710000,C,695.829940,1.0,80.583058,1240.955634,695.829940,695.829940,584.0,2.201829,9.571813


In [28]:
apl = apl[apl["iris1"] == apl["iris2"]]
apl

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl
385,60010000,60010000,0,0,1.0,100.089449,93.000000,C,60010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,100.089449,299.573611,0.000000,60010000,C,100.089449,0.000000,0.000000,299.573611,100.089449,100.089449,93.000000,0.000000,24.378037
157,60020000,60020000,0,0,1.0,61.387834,51.000000,C,60020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,61.387834,263.427392,0.000000,60020000,C,61.387834,0.000000,0.000000,263.427392,61.387834,61.387834,51.000000,0.000000,0.000000
242,60030000,60030000,0,0,1.0,621.316785,640.000000,C,60030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,621.316785,1237.773641,0.000000,60030000,C,621.316785,0.000000,0.000000,1237.773641,621.316785,621.316785,640.000000,0.000000,16.762998
74748,60040101,60040101,0,0,1.0,1699.147290,1763.364267,C,60040101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1699.147290,192073.227204,0.000000,60040101,C,1699.147290,0.000000,0.000000,192073.227204,1699.147290,1699.147290,1763.364267,0.000000,146.297409
74398,60040102,60040102,0,0,1.0,2527.038049,2534.265666,A,60040102,690001.0,BILLAUD,JEAN CHRISTOPHE,6.0,507852.0,7.070016,43.597006,690001_43.597006_7.070016,1.0,1.0,3.000000,2527.038049,153511.652070,1.954249,60040102,A,2527.038049,3.000000,1.954249,153511.652070,2527.038049,2527.038049,2534.265666,1.954249,117.349789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39136,61610105,61610105,0,0,1.0,4042.522443,3584.453834,A,61610105,1437614.0,BRUNET,DIDIER CLEMENT,6.0,1249056.0,7.134178,43.635682,1437614_43.635682_7.134178,1.0,1.0,6.833333,4042.522443,207099.222109,3.299546,61610105,A,4042.522443,6.833333,3.299546,207099.222109,4042.522443,4042.522443,3584.453834,3.299546,153.559212
38772,61610106,61610106,0,0,1.0,3868.825988,3539.143531,A,61610106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,3868.825988,150318.655609,0.000000,61610106,A,3868.825988,0.000000,0.000000,150318.655609,3868.825988,3868.825988,3539.143531,0.000000,112.355171
38423,61610107,61610107,0,0,1.0,2772.707609,2766.654836,A,61610107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,2772.707609,164355.721307,0.000000,61610107,A,2772.707609,0.000000,0.000000,164355.721307,2772.707609,2772.707609,2766.654836,0.000000,122.554038
116175,61620000,61620000,0,0,1.0,875.491796,719.000000,C,61620000,377059.0,DI VINCENZO,DOMINIQUE,6.0,182425.0,7.650077,44.087979,377059_44.087979_7.650077,5.0,0.2,0.200000,875.491796,2082.796507,9.602474,61620000,C,875.491796,0.200000,9.602474,2082.796507,875.491796,875.491796,719.000000,9.602474,41.230952


In [37]:
print(year, specialite, source)
apl["apl"].describe()
#for 2021
#spe10 psmean=95 vs 98, psmedian=101 vs 110, after damir changes psmean=95 psmedian=99
#spe10 pa104-109
#spe3 ps7.2-7.8 pa7.8-8.4
#spe7 ps6.4-6.6 pa 7.7-7.7
#spe10 newpa 98-104
#spe3 newpa 7.4-7.8
#spe7 newpa 6.2-6.4

21 10 PA


count    500.000000
mean      98.474806
std       50.314578
min        0.000000
25%       64.599623
50%      103.694309
75%      138.957982
max      300.187681
Name: apl, dtype: float64

In [30]:
apl[apl["apl"]>400]

,iris1,iris2,km,time,accessibilite_weight,pop_gp,pop,type_iris,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,pop_dest,ap,apl


In [31]:
apl = apl[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp","pop"]]
apl = apl.rename(columns={'iris1': 'iris'})
apl.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp,pop
385,60010000,C,0.0,24.378037,0.000000,1.0,100.089449,299.573611,0.000000,100.089449,93.000000
157,60020000,C,0.0,0.000000,0.000000,1.0,61.387834,263.427392,0.000000,61.387834,51.000000
242,60030000,C,0.0,16.762998,0.000000,1.0,621.316785,1237.773641,0.000000,621.316785,640.000000
74748,60040101,C,0.0,146.297409,0.000000,1.0,1699.147290,192073.227204,0.000000,1699.147290,1763.364267
74398,60040102,A,3.0,117.349789,1.954249,1.0,2527.038049,153511.652070,1.954249,2527.038049,2534.265666


In [32]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '{dept:02d}%'"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2021 "year", 10 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '06%'


,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,60120105,060120105,Centre,6,06012,Beausoleil,2021,10
1,60500000,060500000,Coursegoules,6,06050,Coursegoules,2021,10
2,60720000,060720000,Ilonse,6,06072,Ilonse,2021,10
3,61050101,061050101,Nord,6,06105,Roquefort-les-Pins,2021,10
4,60150000,060150000,Berre-les-Alpes,6,06015,Berre-les-Alpes,2021,10
...,...,...,...,...,...,...,...,...
495,60883403,060883403,Madeleine Supérieure,6,06088,Nice,2021,10
496,60883501,060883501,Crémat-Bellet,6,06088,Nice,2021,10
497,60883601,060883601,Saint-Pierre de Féric,6,06088,Nice,2021,10
498,60883602,060883602,Costière,6,06088,Nice,2021,10


In [33]:
apl = apl.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl = apl[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp","pop", "iris_label", "dept", "code_commune", "commune_label"]]
apl

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label
0,2021,10,60010000,060010000,C,0.000000,24.378037,0.000000,1.0,0.000000,100.089449,299.573611,100.089449,93.000000,Aiglun,6,06001,Aiglun
1,2021,10,60020000,060020000,C,0.000000,0.000000,0.000000,1.0,0.000000,61.387834,263.427392,61.387834,51.000000,Amirat,6,06002,Amirat
2,2021,10,60030000,060030000,C,0.000000,16.762998,0.000000,1.0,0.000000,621.316785,1237.773641,621.316785,640.000000,Andon,6,06003,Andon
3,2021,10,60040101,060040101,C,0.000000,146.297409,0.000000,1.0,0.000000,1699.147290,192073.227204,1699.147290,1763.364267,Les Trois Moulins,6,06004,Antibes
4,2021,10,60040102,060040102,A,3.000000,117.349789,1.954249,1.0,1.954249,2527.038049,153511.652070,2527.038049,2534.265666,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021,10,61610105,061610105,A,6.833333,153.559212,3.299546,1.0,3.299546,4042.522443,207099.222109,4042.522443,3584.453834,Marina,6,06161,Villeneuve-Loubet
496,2021,10,61610106,061610106,A,0.000000,112.355171,0.000000,1.0,0.000000,3868.825988,150318.655609,3868.825988,3539.143531,Vaugrenier,6,06161,Villeneuve-Loubet
497,2021,10,61610107,061610107,A,0.000000,122.554038,0.000000,1.0,0.000000,2772.707609,164355.721307,2772.707609,2766.654836,Plan,6,06161,Villeneuve-Loubet
498,2021,10,61620000,061620000,C,0.200000,41.230952,9.602474,1.0,9.602474,875.491796,2082.796507,875.491796,719.000000,La Brigue,6,06162,La Brigue


In [34]:
# Anomalie = 1 seul medecin avec 1 seul cabinet pour 300 habitants contre 1 seul medecin et 0.33 cabinets
# La commune de sauveur en tinee a un APL > 400 pour les généralistes car l'iris a seulement 300 habitants jeunes et ils sont passés de 1 médecin avec 3 cabinets à 1 médecin a un seul cabinet en 2022
apl[apl["apl"] > 400]


,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,pop,iris_label,dept,code_commune,commune_label


In [35]:
apl.to_csv("apl06.csv", index=False)